In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2 
from albumentations.pytorch import ToTensorV2 # this allows the transform to output a PyTorch Tensor
import albumentations as A
from torch.utils.data import DataLoader
from vector_cv_tools import utils as vutils
from vector_cv_tools import datasets as vdatasets
from vector_cv_tools import transforms as VT
from skimage.color import label2rgb
%matplotlib inline
import torch.nn.functional as F

In [2]:
torch.cuda.empty_cache() 

In [3]:
from lostandfound_loader import lostandfoundLoader
from torch.utils import data

In [4]:
data_path = "/scratch/ssd002/datasets/lostandfound"
import scipy.misc
t_loader = lostandfoundLoader(
        data_path,
        is_transform=True,
        split="train",
        img_size=(128, 128),
        augmentations=None,
    )

trainloader = data.DataLoader(
        t_loader,
        batch_size=4,
        num_workers=2,
        shuffle=True,
    )




Found 1036 train images


In [5]:
def visualize_mvtec(img, mask):
    figure, ax = plt.subplots(1, 2, figsize=(10, 10))
    ax[0].imshow(img)
    ax[0].set_axis_off()
    ax[1].imshow(mask)
    ax[1].set_axis_off()
    plt.tight_layout()
    plt.show()

In [6]:
from convvae import ConvVAE, loss_fn

In [7]:
def show_img_from_tensor(single_img):
    single_img = single_img.permute(1,2,0).cpu().numpy()
    plt.imshow(single_img)
    plt.axis("off")
    plt.show()
    
def evaluate_and_show(model, single_input):
    single_input = single_input.to("cuda")
    with torch.no_grad():
        recon, mu, logvar = model(single_input)
    loss = loss_fn(single_input, recon, mu , logvar)
    print("Loss is: ", loss.item())
    show_img_from_tensor(recon.squeeze(0))

In [8]:
#basic_transform = VT.ComposeMVTecTransform([A.Resize(128, 128), A.ToFloat(max_value=255), ToTensorV2()])
#test_dset = vdatasets.MVTec(MVTEC_ROOT_DIR, split="test", mask_only= True ,obj_types=["toothbrush"], transforms = basic_transform)
#train_dset = vdatasets.MVTec(MVTEC_ROOT_DIR, split="train", transforms = basic_transform, obj_types=["toothbrush"])

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm
from albumentations.pytorch import ToTensorV2
import albumentations as A
from torchvision.utils import save_image
from vector_cv_tools import utils as vutils
from vector_cv_tools import transforms as VT
from vector_cv_tools import datasets as vdatasets
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2 
from albumentations.pytorch import ToTensorV2 # this allows the transform to output a PyTorch Tensor
import albumentations as A
from torch.utils.data import DataLoader
from vector_cv_tools import utils as vutils
from vector_cv_tools import datasets as vdatasets
from vector_cv_tools import transforms as VT
from skimage.color import label2rgb
%matplotlib inline


In [9]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm
from albumentations.pytorch import ToTensorV2
import albumentations as A
from torchvision.utils import save_image
from vector_cv_tools import utils as vutils
from vector_cv_tools import transforms as VT
from vector_cv_tools import datasets as vdatasets
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2 
from albumentations.pytorch import ToTensorV2 # this allows the transform to output a PyTorch Tensor
import albumentations as A
from torch.utils.data import DataLoader
from vector_cv_tools import utils as vutils
from vector_cv_tools import datasets as vdatasets
from vector_cv_tools import transforms as VT
from skimage.color import label2rgb
%matplotlib inline
def apply_random_mask(img):
        """Randomly masks image"""
        img_size=img.shape[0]
        mask_size=50
        y1 = np.random.randint(0, img.shape[0]- mask_size, 1)
        x1 = np.random.randint(0, img.shape[1]- mask_size, 1)
        y2, x2 = y1 + mask_size, x1 + mask_size
        y1=int(y1)
        y2=int(y2)
        x1=int(x1)
        x2=int(x2)
        masked_part = img[y1:y2, x1:x2]
        masked_img = img

        masked_img[y1:y2, x1:x2] = 0
        return masked_img, masked_part

In [11]:
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
#plt.imshow(mpimg.imread('Sample.png'))
#img=mpimg.imread('Sample.png')
#print(img.shape)
#masked_img, masked_part=apply_random_mask(img)
#plt.imshow(masked_img)

In [10]:
def masking_image(img):
    mask_size=20
    x1 = np.random.randint(0, img.shape[2]- mask_size, 1)
    x1=int(x1)
    img[:,:,x1:x1+mask_size,x1:x1+mask_size]=1
    return img

In [12]:
#training model without loading the weights
from convvae import  loss_fn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = ConvVAE().to(device)
model = torch.nn.DataParallel(model)

lr = 3e-5
optimizer = Adam(model.parameters(), lr=lr)
num_epochs = 40
for epoch in tqdm(range(num_epochs)):
        train_losses = []
        for i, (data, _) in enumerate(trainloader):       
            #Feeding model with the masked data 
            data = data.to(device)  
            optimizer.zero_grad()
            data_masked=masking_image(data)
            recon, mu, logvar = model(data_masked)
            loss1 = loss_fn(data, recon, mu, logvar)
            loss1.backward()
            optimizer.step()
            
            #Feeding model with the original data
            optimizer.zero_grad()
            recon2, mu, logvar = model(data)
            loss2 = loss_fn(data, recon2, mu, logvar)
            loss2.backward()
            optimizer.step()
            train_losses.append((loss1.item()+ loss2.item())/2)
            if i % 200 == 0:
                save_image(recon2.detach().cpu(), f"Project/2000{epoch+1}_{i}_NewRound2_NEW.png")
        mean_loss = sum(train_losses) / len(train_losses) if (
            len(train_losses) > 0) else 0
        print(f"train loss at epoch {epoch+1} is {mean_loss}")
        if (epoch % 4) == 0:
            torch.save(model.state_dict(), f"Project/2000NEW_model_{epoch}_NewRound2.pt")


  0%|          | 0/40 [00:00<?, ?it/s]

train loss at epoch 1 is 1476.0000097797645



  5%|▌         | 2/40 [04:07<1:18:30, 123.96s/it]

train loss at epoch 2 is 1027.6740581262065



  8%|▊         | 3/40 [06:09<1:16:12, 123.57s/it]

...

train loss at epoch 38 is 420.1526508699513



 98%|█████████▊| 39/40 [1:19:57<02:02, 122.65s/it]

train loss at epoch 39 is 406.61616239216335



100%|██████████| 40/40 [1:21:59<00:00, 122.99s/it]

train loss at epoch 40 is 406.1442940612557


In [13]:
#training model without loading the weights
#from convvae import  loss_fn
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#model = ConvVAE().to(device)
#model = torch.nn.DataParallel(model)

Path="Project/2000NEW_model_32_NewRound2.pt"
model.load_state_dict(torch.load(Path))

lr = 3e-5
optimizer = Adam(model.parameters(), lr=lr)
num_epochs = 21
for epoch in tqdm(range(num_epochs)):
        train_losses = []
        for i, (data, _) in enumerate(trainloader):       
            #Feeding model with the masked data 
            data = data.to(device)  
            optimizer.zero_grad()
            data_masked=masking_image(data)
            recon, mu, logvar = model(data_masked)
            loss1 = loss_fn(data, recon, mu, logvar)
            loss1.backward()
            optimizer.step()
            
            #Feeding model with the original data
            optimizer.zero_grad()
            recon2, mu, logvar = model(data)
            loss2 = loss_fn(data, recon2, mu, logvar)
            loss2.backward()
            optimizer.step()
            train_losses.append((loss1.item()+ loss2.item())/2)
            if i % 200 == 0:
                save_image(recon2.detach().cpu(), f"Project/2000{epoch+1+40}_{i}_NewRound2_NEW.png")
        mean_loss = sum(train_losses) / len(train_losses) if (
            len(train_losses) > 0) else 0
        print(f"train loss at epoch {epoch+1} is {mean_loss}")
        if (epoch % 4) == 0:
            torch.save(model.state_dict(), f"Project/2000NEW_model_{epoch+40}_NewRound2.pt")


  0%|          | 0/21 [00:00<?, ?it/s]

train loss at epoch 1 is 444.7702506518272



 10%|▉         | 2/21 [04:08<39:29, 124.69s/it]

train loss at epoch 2 is 440.8149797005083



 14%|█▍        | 3/21 [06:10<37:12, 124.00s/it]

...


 90%|█████████ | 19/21 [38:57<04:05, 122.82s/it]

train loss at epoch 19 is 351.00312996647074



 95%|█████████▌| 20/21 [40:59<02:02, 122.65s/it]

train loss at epoch 20 is 341.63709801044246
train loss at epoch 21 is 341.18578435831546



100%|██████████| 21/21 [43:04<00:00, 123.06s/it]


In [ ]:
from convvae import  loss_fn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Path="Project/2000NEW_model_60_NewRound2.pt"
model = ConvVAE().to(device)
model = torch.nn.DataParallel(model)
model.load_state_dict(torch.load(Path))
from convvae import  loss_fn

for i, (data, _) in enumerate(trainloader): 
    if (i<11):
        recon2, mu, logvar = model(data)
        save_image(data.detach().cpu(), f"Samples/Real_{i}.png")
        save_image(recon2.detach().cpu(), f"Samples/Pre_{i}.png")